# 공공데이터 포털 - 기상청_단기예보 ((구)_동네예보) 조회서비스 Open API 사용 기초 예제

# 초단기실황조회 API (getUltraSrtNcst) 사용법 살펴보기

In [17]:
decoding_일반_인증키 = 'I6gVrtvyqmw1ST2WA4TpH2GeNRB1rG4yFsPDYBlCeGTjwPtNuMDHvglULEYxgf5Pa0S1R6QOA313QR0oo3/THw=='

In [18]:
from datetime import datetime

def get_current_date():
  current_date = datetime.now()
  return current_date.strftime("%Y%m%d")

def get_current_hour():
  now = datetime.now()
  return now.strftime("%H%M")

In [19]:
import requests

url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst'
params = {
    'serviceKey' : decoding_일반_인증키,
    'pageNo': '1',
    'numOfRows': '1000',
    'dataType': 'JSON',
    'base_date' : get_current_date(),
    'base_time' : get_current_hour(),
    'nx' : '55',
    'ny' : '127'
}

response = requests.get(url, params=params)

In [20]:
import json
result = json.loads(response.text)
result

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL_SERVICE'},
  'body': {'dataType': 'JSON',
   'items': {'item': [{'baseDate': '20250224',
      'baseTime': '1800',
      'category': 'PTY',
      'nx': 55,
      'ny': 127,
      'obsrValue': '0'},
     {'baseDate': '20250224',
      'baseTime': '1800',
      'category': 'REH',
      'nx': 55,
      'ny': 127,
      'obsrValue': '57'},
     {'baseDate': '20250224',
      'baseTime': '1800',
      'category': 'RN1',
      'nx': 55,
      'ny': 127,
      'obsrValue': '0'},
     {'baseDate': '20250224',
      'baseTime': '1800',
      'category': 'T1H',
      'nx': 55,
      'ny': 127,
      'obsrValue': '0.4'},
     {'baseDate': '20250224',
      'baseTime': '1800',
      'category': 'UUU',
      'nx': 55,
      'ny': 127,
      'obsrValue': '0.5'},
     {'baseDate': '20250224',
      'baseTime': '1800',
      'category': 'VEC',
      'nx': 55,
      'ny': 127,
      'obsrValue': '204'},
     {'baseDate': '20250224',
    

In [21]:
weather_enum = {
    "0": "맑음",
    "1": "비",
    "2": "비/눈",
    "3": "눈",
    "4": "빗방울",
    "5": "빗방울눈날림",
    "6": "눈날림"
}

In [24]:
def parse_weather_infomation(result):
  for item in result['response']['body']['items']['item']:
    if item['category'] == 'PTY':
      weather = weather_enum[item['obsrValue']]
    if item['category'] == 'REH':
      습도 = item['obsrValue']
    if item['category'] == 'RN1':
      한시간강수량 = item['obsrValue']
    if item['category'] == 'T1H':
      기온 = item['obsrValue']

  return_string = ''
  return_string += f'날씨 : {weather}\n'
  return_string += f'습도 : {습도}\n'
  return_string += f'1시간강수량 : {한시간강수량}\n'
  return_string += f'기온 : {기온}\n'

  return return_string

In [25]:
weather_info = parse_weather_infomation(result)
print(weather_info)

날씨 : 맑음
습도 : 57
1시간강수량 : 0
기온 : 0.4



# 하나의 함수로 묶기

In [26]:
def get_weather_infomation(decoding_일반_인증키, x_position, y_position):
  url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst'
  params = {
      'serviceKey' : decoding_일반_인증키,
      'pageNo': '1',
      'numOfRows': '1000',
      'dataType': 'JSON',
      'base_date' : get_current_date(),
      'base_time' : get_current_hour(),
      'nx' : x_position,
      'ny' : y_position
  }

  response = requests.get(url, params=params)
  weather_info = parse_weather_infomation(json.loads(response.text))

  return weather_info

In [27]:
x_position = '55'
y_position = '127'

weather_info = get_weather_infomation(decoding_일반_인증키, x_position, y_position)
print(weather_info)

날씨 : 맑음
습도 : 71
1시간강수량 : 0
기온 : -0.2



# 예보지점 X 좌표, 예보지점 Y 좌표 얻기

In [28]:
import pandas as pd

In [31]:
file_path = '기상청41_단기예보 조회서비스_오픈API활용가이드_격자_위경도(2411).xlsx'

In [ ]:
df = pd.read_excel(file_path)
df

In [33]:
df.columns

Index(['구분', '행정구역코드', '1단계', '2단계', '3단계', '격자 X', '격자 Y', '경도(시)', '경도(분)',
       '경도(초)', '위도(시)', '위도(분)', '위도(초)', '경도(초/100)', '위도(초/100)', '위치업데이트'],
      dtype='object')

In [47]:
def get_grid_coordinates(df, level1, level2=None, level3=None):
  query = f"`1단계` == '{level1}'"
  if level2 is not None:
    query += f" and `2단계` == '{level2}'"
  if level3 is not None:
    query += f" and `3단계` == '{level3}'"

  print(query)
  result = df.query(query)
  if not result.empty:
    grid_x = result.iloc[0]['격자 X']
    grid_y = result.iloc[0]['격자 Y']
    return grid_x, grid_y
  else :
    return None, None

In [48]:
# 함수 사용 예시
level1 = '서울특별시'
level2 = '종로구'
level3 = '청운효자동'

grid_x, grid_y = get_grid_coordinates(df, level1, level2, level3)
print(f"격자 X: {grid_x}, 격자 Y: {grid_y}")

`1단계` == '서울특별시' and `2단계` == '종로구' and `3단계` == '청운효자동'
격자 X: 60, 격자 Y: 127


In [49]:
level1 = '서울특별시'

grid_x, grid_y = get_grid_coordinates(df, level1)
print(f"격자 X: {grid_x}, 격자 Y: {grid_y}")

`1단계` == '서울특별시'
격자 X: 60, 격자 Y: 127
